# Alternative 1 - On raw signal

In [191]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf

In [34]:
df = pd.read_csv("./datasets/df_raw_features.tar.gz")
df.head()

,pid,time,id,sort,sleep_phase,act_0,act_1,act_2,act_3,act_4,...,hr_3,hr_4,hr_5,hr_6,hr_7,hr_8,hr_9,hr_10,hr_11,hr_12
0,1,29,"(1, 0)",0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,59,"(1, 1)",1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,89,"(1, 2)",2,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,119,"(1, 3)",3,0.0,0.0,0.0,0.0,0.0,0.0,...,75.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,149,"(1, 4)",4,0.0,0.0,0.0,0.0,0.0,85.0,...,75.0,80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [193]:
df["sleep_phase"].unique()
# 0 -> Wake
# 1 -> phase 1 (light sleep)
# 2 -> phase 2 (deep sleep N1)
# 3 -> phase 3 (deep sleep N2)
# 4 -> NREM    ()
# 5 -> REM

array([0., 1., 2., 5., 3., 4.])

In [194]:
df["bin_sleep_phase"] = df["sleep_phase"] > 0


- Model input (S, 2, 13): 
                       [
                        [hr_0, hr_1, hr_2   ....hr_12]
                        [act_0, act_1, act_2....act_12]
                       ]
- Model output:
    - (S, 1) (bin sleep phase)
    - (S, X) (sleep_)



### Get XY from dataframe

In [196]:
def generate_XY(df, ycol="bin_sleep_phase"):
    
    hr_cols = [k for k in df.keys() if k.startswith("hr_")]
    act_cols = [k for k in df.keys() if k.startswith("act_")]

    hr = df[hr_cols].values
    act = df[act_cols].values
    
    Y  = df[ycol].values.reshape(-1, 1)

    X = np.stack((act,hr))
    X = X.transpose(1,0,2)
    
    return X, Y


In [197]:
df_XY = df.groupby("pid").apply(lambda x: generate_XY(x))
df_XY.head()

pid
1     ([[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], [...
16    ([[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], [...
21    ([[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0...
28    ([[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], [...
33    ([[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], [...
dtype: object

In [198]:
idx = 2
df_XY.iloc[idx][0].shape, df_XY.iloc[idx][1].shape

((1075, 2, 13), (1075, 1))

In [199]:
xs, ys = [], []
for row_id, (x, y) in df_XY.items():
    xs.append(x)
    ys.append(y)
    
xs = np.array(xs, dtype=object)
ys = np.array(ys, dtype=object)


In [200]:
# Now we can create a simple trainset from the dataset making sure that
# data from one subject is NOT at the same time in the training and in the test sets
subjects_train_idx = [1, 2, 3, 4]    
np.vstack(xs[subjects_train_idx]).shape, np.vstack(ys[subjects_train_idx]).shape

((3504, 2, 13), (3504, 1))

In [201]:
subjects_train_idx = range(100)
X_train = np.vstack(xs[subjects_train_idx])
Y_train = np.vstack(ys[subjects_train_idx])

subjects_test_idx = range(100, 150)
X_val = np.vstack(xs[subjects_test_idx])
Y_val = np.vstack(ys[subjects_test_idx])

subjects_test_idx = range(150, 200)
X_test = np.vstack(xs[subjects_test_idx])
Y_test = np.vstack(ys[subjects_test_idx])


### Evaluate a few models

In [202]:
def simple_dense_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(12, input_shape=(2, 13), activation='relu'))
    model.add(tf.keras.layers.Dense(8, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

simple_model = simple_dense_model()
  

In [204]:
early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
history = simple_model.fit(X_train, Y_train, 
                           validation_data=(X_val, Y_val), 
                           epochs=50, 
                           batch_size=8,
                           callbacks=[early_stop_callback])


Epoch 1/50
12858/12858 [==============================] - 59s 5ms/step - loss: 0.4959 - accuracy: 0.7731 - val_loss: 0.4672 - val_accuracy: 0.7991
Epoch 2/50
12858/12858 [==============================] - 64s 5ms/step - loss: 0.4926 - accuracy: 0.7755 - val_loss: 0.4611 - val_accuracy: 0.8016
Epoch 3/50
12858/12858 [==============================] - 69s 5ms/step - loss: 0.4904 - accuracy: 0.7768 - val_loss: 0.4599 - val_accuracy: 0.7995
Epoch 4/50
12858/12858 [==============================] - 75s 6ms/step - loss: 0.4892 - accuracy: 0.7779 - val_loss: 0.4539 - val_accuracy: 0.8039
Epoch 5/50
12858/12858 [==============================] - 61s 5ms/step - loss: 0.4881 - accuracy: 0.7790 - val_loss: 0.4555 - val_accuracy: 0.8049
Epoch 6/50
12858/12858 [==============================] - 51s 4ms/step - loss: 0.4879 - accuracy: 0.7790 - val_loss: 0.4514 - val_accuracy: 0.8058
Epoch 7/50
12858/12858 [==============================] - 47s 4ms/step - loss: 0.4867 - accuracy: 0.7798 - val_loss: 0

In [205]:
simple_model.evaluate(x=X_test, y=Y_test)

1603/1603 [==============================] - 3s 2ms/step - loss: 0.4985 - accuracy: 0.7755


[0.4985067546367645, 0.7754501104354858]

In [185]:
def cnn_lstm_model(cnn_d = 10, lstm_d = 4):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv1D(cnn_d, kernel_size=(3,), padding='same'))
    model.add(tf.keras.layers.BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9))
    model.add(tf.keras.layers.Activation(tf.nn.relu))
    
    model.add(tf.keras.layers.Dropout(0.1))
    model.add(tf.keras.layers.LSTM(lstm_d, return_sequences=False))
    model.add(tf.keras.layers.Dense(1, activation="sigmoid", name='output'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

cnnlstm_model = cnn_lstm_model()

In [186]:
early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
history = cnnlstm_model.fit(X_train, Y_train, 
                           validation_data=(X_val, Y_val), 
                           epochs=50, 
                           batch_size=8,
                           callbacks=[early_stop_callback])

Epoch 1/50


2022-09-04 15:17:35.898353: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101


12858/12858 [==============================] - 80s 6ms/step - loss: 0.4392 - accuracy: 0.8155 - val_loss: 0.3893 - val_accuracy: 0.8397
Epoch 2/50
12858/12858 [==============================] - 74s 6ms/step - loss: 0.4217 - accuracy: 0.8223 - val_loss: 0.3886 - val_accuracy: 0.8431
Epoch 3/50
12858/12858 [==============================] - 76s 6ms/step - loss: 0.4166 - accuracy: 0.8238 - val_loss: 0.3829 - val_accuracy: 0.8416
Epoch 4/50
12858/12858 [==============================] - 76s 6ms/step - loss: 0.4138 - accuracy: 0.8255 - val_loss: 0.3810 - val_accuracy: 0.8439
Epoch 5/50
12858/12858 [==============================] - 78s 6ms/step - loss: 0.4121 - accuracy: 0.8260 - val_loss: 0.3769 - val_accuracy: 0.8454
Epoch 6/50
12858/12858 [==============================] - 78s 6ms/step - loss: 0.4105 - accuracy: 0.8265 - val_loss: 0.3936 - val_accuracy: 0.8436
Epoch 7/50
12858/12858 [==============================] - 79s 6ms/step - loss: 0.4099 - accuracy: 0.8257 - val_loss: 0.3876 - val

In [187]:
cnnlstm_model.evaluate(x=X_test, y=Y_test)

1603/1603 [==============================] - 3s 2ms/step - loss: 0.4965 - accuracy: 0.7755


[0.49647629261016846, 0.7755085825920105]